In [34]:
import os
import pickle
import click

In [35]:
import pandas as pd

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

In [37]:
import mlflow
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature

In [38]:
HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models"
RF_PARAMS = [
    'max_depth', 
    'n_estimators', 
    'min_samples_split', 
    'min_samples_leaf', 
    'random_state'
]

In [39]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.sklearn.autolog()

2024/05/27 12:26:36 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


In [40]:
DATA_DIR_PATH = "./output/"

In [41]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [42]:
def run_train(data_path: str):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)

    return rmse

## Homework

The goal of this homework is to get familiar with MLflow, the tool for experiment tracking and 
model management.

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

```plain
mlflow                    2.13.0                   pypi_0    pypi
```

## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the duration of each trip. 

Download the data for January, February and March 2023 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](homework) to preprocess the data.

The script will:

* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2023 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* **4** (answer)
* 7


```bash
python3 preprocess_data.py --raw_data_path ../data/green/ --dest_path ../output
```

```ls ../output
dv.pkl  test.pkl  train.pkl  val.pkl
```

## Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `min_samples_split` parameter:

* **2** answer
* 4
* 8
* 10


In [43]:
train_data_path = os.path.join(DATA_DIR_PATH, "train.pkl")
val_data_path = os.path.join(DATA_DIR_PATH, "val.pkl")
test_data_path = os.path.join(DATA_DIR_PATH, "test.pkl")

X_train, y_train = load_pickle(train_data_path)
X_val, y_val = load_pickle(val_data_path)
X_test, y_test = load_pickle(test_data_path)


with mlflow.start_run():

    mlflow.set_tag("developer", "jelambrar")

    mlflow.log_param("train_data_path", train_data_path)
    mlflow.log_param("val_data_path", val_data_path)

    model_params = { "max_depth": 10, "random_state": 0 }

    mlflow.log_param("max_depth", model_params["max_depth"])
    mlflow.log_param("random_state", model_params["random_state"])

    rf = RandomForestRegressor(**model_params)
    rf.fit(X_train, y_train)
    
    y_pred_val = rf.predict(X_val)
    rmse_val = root_mean_squared_error(y_val, y_pred_val)
    mlflow.log_metric("rmse_val", rmse_val)


    y_pred_test = rf.predict(X_test)
    rmse_test = root_mean_squared_error(y_test, y_pred_test)
    mlflow.log_metric("rmse_test", rmse_test)

    signature = infer_signature(X_test, y_pred_test)

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=rf,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="sk-learn-random-forest-reg-model",
    )



2024/05/27 12:26:37 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
Successfully registered model 'sk-learn-random-forest-reg-model'.
2024/05/27 12:26:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: sk-learn-random-forest-reg-model, version 1
Created version '1' of model 'sk-learn-random-forest-reg-model'.


![](media/Screenshot%20from%202024-05-27%2012-27-53.png)

## Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to:

* launch the tracking server on your local machine,
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

## Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `hyperopt`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

* 4.817
* **5.335** (answer)
* 5.818
* 6.336


In [44]:
import numpy as np

In [45]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope

In [46]:
def run_optimization(num_trials:int ):

    def objective(params):

        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        return {'loss': rmse, 'status': STATUS_OK}


    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

    rstate = np.random.default_rng(42)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )

run_optimization(15)

  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

2024/05/27 12:50:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6f8b7193fa874f7db75a242d2672d96e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow

2024/05/27 12:50:36 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



  7%|▋         | 1/15 [00:08<01:52,  8.05s/trial, best loss: 5.370086069268862]

2024/05/27 12:50:43 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ba92fb583fbc40669b2e16c607836b3f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow

2024/05/27 12:50:43 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



 13%|█▎        | 2/15 [00:10<01:04,  4.98s/trial, best loss: 5.370086069268862]

...

 87%|████████▋ | 13/15 [01:13<00:10,  5.01s/trial, best loss: 5.335419588556921]

2024/05/27 12:51:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '5ead1c9d971e4946a63470c6531a30df', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow

2024/05/27 12:51:49 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



 93%|█████████▎| 14/15 [01:19<00:05,  5.37s/trial, best loss: 5.335419588556921]

2024/05/27 12:51:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '21c9c85077e3429799c04f698dde659a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow

2024/05/27 12:51:55 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'



100%|██████████| 15/15 [01:25<00:00,  5.71s/trial, best loss: 5.335419588556921]


## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 5.060
* **5.567** (answer)
* 6.061
* 6.568



In [83]:
def train_and_log_model(params):

    with mlflow.start_run():
        for param in RF_PARAMS:
            # print(param)
            params[param] = int(params[param])
        
        print(params)

        for kp, vp in params.items():
            if vp in ["True", "False"]:
                params[kp] = bool(vp)
                continue
            
            if vp == "None":
                params[kp] = None
            
            if kp == "verbose":
                try:
                    params[kp] = int(float(vp))
                except:
                    pass
                finally:
                    continue                

            if type(vp) == str:
                try:
                    params[kp] = float(vp)
                except:
                    continue 


        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)

        # Evaluate model on the validation and test sets
        val_rmse = root_mean_squared_error(y_val, rf.predict(X_val))
        mlflow.log_metric("rmse_val", val_rmse)
        test_rmse = root_mean_squared_error(y_test, rf.predict(X_test))
        mlflow.log_metric("rmse_test", test_rmse)

In [84]:
TOP_N = 5

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")
# Retrieve the top_n model runs and log the models
experiment = client.get_experiment_by_name("random-forest-best-models")
runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=TOP_N,
    order_by=["metrics.rmse ASC"]
)
for run in runs:
    train_and_log_model(params=run.data.params)

# Select the model with the lowest test RMSE
# experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
# best_run = client.search_runs( ...  )[0]

2024/05/27 14:30:51 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


{'bootstrap': 'True', 'ccp_alpha': '0.0', 'criterion': 'squared_error', 'max_depth': 20, 'max_features': '1.0', 'max_leaf_nodes': 'None', 'max_samples': 'None', 'min_impurity_decrease': '0.0', 'min_samples_leaf': 1, 'min_samples_split': 9, 'min_weight_fraction_leaf': '0.0', 'monotonic_cst': 'None', 'n_estimators': 19, 'n_jobs': 'None', 'oob_score': 'False', 'random_state': 42, 'verbose': '0', 'warm_start': 'False'}


/home/jelambrar/anaconda3/envs/mlops_hw2/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
2024/05/27 14:30:59 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


{'bootstrap': 'True', 'ccp_alpha': '0.0', 'criterion': 'squared_error', 'max_depth': 18, 'max_features': '1.0', 'max_leaf_nodes': 'None', 'max_samples': 'None', 'min_impurity_decrease': '0.0', 'min_samples_leaf': 1, 'min_samples_split': 6, 'min_weight_fraction_leaf': '0.0', 'monotonic_cst': 'None', 'n_estimators': 13, 'n_jobs': 'None', 'oob_score': 'False', 'random_state': 42, 'verbose': '0', 'warm_start': 'False'}


/home/jelambrar/anaconda3/envs/mlops_hw2/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
2024/05/27 14:31:06 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


...

/home/jelambrar/anaconda3/envs/mlops_hw2/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
2024/05/27 14:31:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


{'bootstrap': 'True', 'ccp_alpha': '0.0', 'criterion': 'squared_error', 'max_depth': 7, 'max_features': '1.0', 'max_leaf_nodes': 'None', 'max_samples': 'None', 'min_impurity_decrease': '0.0', 'min_samples_leaf': 1, 'min_samples_split': 7, 'min_weight_fraction_leaf': '0.0', 'monotonic_cst': 'None', 'n_estimators': 41, 'n_jobs': 'None', 'oob_score': 'False', 'random_state': 42, 'verbose': '0', 'warm_start': 'False'}


2024/05/27 14:31:16 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


{'bootstrap': 'True', 'ccp_alpha': '0.0', 'criterion': 'squared_error', 'max_depth': 19, 'max_features': '1.0', 'max_leaf_nodes': 'None', 'max_samples': 'None', 'min_impurity_decrease': '0.0', 'min_samples_leaf': 2, 'min_samples_split': 2, 'min_weight_fraction_leaf': '0.0', 'monotonic_cst': 'None', 'n_estimators': 11, 'n_jobs': 'None', 'oob_score': 'False', 'random_state': 42, 'verbose': '0', 'warm_start': 'False'}


/home/jelambrar/anaconda3/envs/mlops_hw2/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(



## Submit the results

* Submit your results here: https://courses.datatalks.club/mlops-zoomcamp-2024/homework/hw2
* You can submit your solution multiple times. In this case, only the last submission will be used
* If your answer doesn't match options exactly, select the closest one
